In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
import numpy as np

In [2]:
file_path = '/kaggle/input/flickr8k/captions.txt'
image_urls=[]
captions=[]
with open(file_path, 'r') as file:
    for line in file:
        image_urls.append(line.strip().split(",")[0])
        captions.append(line.strip().split(",")[1])
image_urls.pop(0)
captions.pop(0)
print(len(image_urls),len(captions))

40455 40455


In [3]:
tokenizer = Tokenizer()
def Tokenize(sentence,tokenizer):
    tokenizer.fit_on_texts([sentence])
    sequence = tokenizer.texts_to_sequences([sentence])
    sequence_length = 50  
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=sequence_length)
    return padded_sequence

In [4]:
model = VGG16(weights='imagenet')
def encoder(img,model):
    img = image.load_img("/kaggle/input/flickr8k/Images/"+img, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    image_features=model.predict(img_array)
    return image_features

553467096/553467096 [==============================] - 15s 0us/step


In [5]:
def decoder(image_features,caption_embeddings):
    v = tf.random.normal(shape=(1,1000))
    Wh = tf.random.normal(shape=(1,1000))
    Ws = tf.random.normal(shape=(1,1000))
    battn = tf.random.normal(shape=(1,1000))
    decoder_state=tf.zeros((1,1000))
    LSTM_model = tf.keras.Sequential([ Embedding(input_dim=10000 ,output_dim=20, input_length=50),
    tf.keras.layers.LSTM(64, input_shape=(1, 50,20)),
    tf.keras.layers.Dense(1000)])
    captions_features=LSTM_model(caption_embeddings)
    e_t_i = v * tf.nn.tanh(Wh * image_features+ Ws*decoder_state+  battn)
    attention_distributon=tf.nn.softmax(e_t_i)
    context_vector=attention_distributon*image_features
    combined=captions_features+attention_distributon
    output = tf.keras.layers.Dense(1000, activation='softmax')(combined)
    epsilon = 1e-15
    y_pred = np.clip(output, epsilon, 1 - epsilon)
    loss = -np.sum(captions_features* np.log(y_pred))
    num_samples = len(captions_features)
    loss /= num_samples
    print(loss)
    decoder_state=(decoder_state+context_vector)/2

In [6]:
image_features=encoder(image_urls[0],model)
tokens=Tokenize(captions[0],tokenizer)
decoder(image_features,tokens)

1/1 [==============================] - 1s 589ms/step
0.4205624461174011
